In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2
import shutil
import os

In [10]:
#extraer info de imagenes sin hallazgos para adjuntarlas al dataset
todo= pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Todas.csv")
bboxes=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/bboxes.csv")
data=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Data.csv")

In [6]:
NoOpacities=data[data['Finding Labels']=='No Finding']

In [8]:
paths=NoOpacities['Image Index']
labels=NoOpacities['Finding Labels']

In [9]:
df = pd.DataFrame({'id': paths, 'label': labels})

In [11]:
df.to_csv('/media/deadwolf/BEC4-9B89/chext14/NoOp.csv', index=False)

In [3]:
info= pd.read_csv("bboxes.csv")
info.head()

,id,label,bboxes
0,00013118_008.png,Atelectasis,"[[225.084745762712,547.019216763771,86.7796610..."
1,00014716_007.png,Atelectasis,"[[686.101694915254,131.543498411017,185.491525..."
2,00029817_009.png,Atelectasis,"[[221.830508474576,317.053115068856,155.118644..."
3,00014687_001.png,Atelectasis,"[[726.237288135593,494.951420153602,141.016949..."
4,00017877_001.png,Atelectasis,"[[660.067796610169,569.78078654661,200.6779661..."
